In [1]:
import xarray as xr
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.76 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46239,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33975,Total threads: 1
Dashboard: http://127.0.0.1:44707/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:38155,


In [2]:
# Example: Access ERA5 data
path = 'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr'

ds = xr.open_dataset(path, engine='zarr').chunk('auto')
ds

<xarray.Dataset> Size: 41TB
Dimensions:                                           (time: 92044,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * time                                              (time) datetime64[ns] 736kB ...
  * latitude                                          (latitude) float32 3kB ...
  * longitude                                         (longitude) float32 6kB ...
  * level                                             (level) int64 104B 50 ....
Data variables: (12/38)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 382GB dask.array<chunksize=(322, 322, 322), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 382GB dask.array<chunksize=(322, 322, 322), meta=np.ndarray>
    10m_wind_speed                                    (time, latitude, longitude) float32 382GB dask.array<chunksize=(322, 322, 322), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 382GB dask.array<chunksize=(322, 322, 322), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    type_of_low_vegetation                            (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    u_component_of_wind                               (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(137, 13, 137, 137), meta=np.ndarray>
    v_component_of_wind                               (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(137, 13, 137, 137), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(137, 13, 137, 137), meta=np.ndarray>
    wind_speed                                        (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(137, 13, 137, 137), meta=np.ndarray>

In [ ]:
t2m = ds['2m_temperature']
t2m 

In [ ]:
t2m.isel(time=-2).plot(cmap='magma')

In [ ]:
# Quickly swap over to the dask dashboard to watch
t2m.isel(time=slice(0, 322)).mean(dim='time').plot(cmap='magma')

In [ ]:
tucson_lat = 32.2540
tucson_lon = 360.0 - 110.9742

year_number = 7
timesteps_per_day = 4
days_per_year = 365
start = year_number * timesteps_per_day * 365
end = (year_number + 1) * timesteps_per_day  * 365

tucson_t2m = t2m.isel(time=slice(start, end)).sel(
    latitude=tucson_lat,
    longitude=tucson_lon,
    method='nearest'
)


In [ ]:
(tucson_t2m - 273).plot()

In [ ]:
start_year = 10
number_years = 10
timesteps_per_day = 4
days_per_year = 365
start = start_year * timesteps_per_day * 365
end = (start_year + number_years) * timesteps_per_day  * 365

tucson_t2m = t2m.isel(time=slice(start, end, 7)).sel(
    latitude=tucson_lat,
    longitude=tucson_lon,
    method='nearest'
)

In [ ]:
tucson_t2m_climatology = tucson_t2m.groupby(tucson_t2m.time.dt.month).mean()

In [ ]:
tucson_t2m_climatology.plot()

In [ ]:
start = 0
end = -1
step = 77
tucson_t2m = t2m.isel(time=slice(start, end, step)).sel(
    latitude=tucson_lat,
    longitude=tucson_lon,
    method='nearest'
)

tucson_t2m_climatology = tucson_t2m.groupby(tucson_t2m.time.dt.month).mean()


In [ ]:

tucson_t2m_climatology.plot()